In [1]:
import pandas as pd
import zipfile
from kaggle.api.kaggle_api_extended import KaggleApi

In [2]:
api = KaggleApi()
api.authenticate()

In [3]:
api.dataset_download_file(dataset='anmolkumar/health-insurance-cross-sell-prediction',
                         file_name='test.csv', path='../zip')

True

In [4]:
with zipfile.ZipFile('../zip/test.csv.zip','r') as zipref:
    zipref.extractall(path='../00-Data')

In [39]:
df_teste = pd.read_csv('../00-Data/test.csv')

In [31]:
import inflection
import bisect
import pandas as pd
import joblib as jb
import inflection
import bisect

In [32]:
class insuranse_cross_sell(object):
    def __init__(self):
        self.home_path          = 'D:/01-DataScience/04-Projetos/00-Git/insurance-cross-sell/'
        self.age_stage_encoder  = jb.load(open(self.home_path + '03-Parameter/age_stage.pkl.z', 'rb'))
    
    def data_cleaning(self, df):
        # snakecase
        snakecase = lambda col: inflection.underscore(col)
        new_columns = list(map(snakecase, df.columns))

        # rename
        df.columns = new_columns

        return df
    
    def age_stage(self, num, breakpoints=[10, 20, 30, 45, 60, 70, 80, 120], result='01234567'):
        i = bisect.bisect(breakpoints, num-1)
        age_mapping = {
            0: 'Child',
            1: 'Teenager',
            2: 'Young',
            3: 'Adult',
            4: 'Midlife',
            5: 'Senior',
            6: 'Mature Adulthood',
            7: 'Late Adulthood'
        }
        return age_mapping[i]

    
    def feature_engineering(self, df):
        # Age Stage
        df['age_stage'] = df['age'].apply(lambda row: insuranse_cross_sell.age_stage(self,row))
        
        # vehicle age
        vehicle_age_mapping = {'< 1 Year':0,
                               '1-2 Year':1,
                               '> 2 Years':2}

        df['vehicle_age'] = df['vehicle_age'].map(vehicle_age_mapping)
        
        # vehicle demage versus license
        vehicle_damage_mapping = {'No':0,
                                  'Yes':1}

        df['vehicle_damage'] = df['vehicle_damage'].map(vehicle_damage_mapping)
        df['vehicle_damage_license'] = df.apply(lambda row: -(row['vehicle_damage'] + row['driving_license'])**2\
                                                 if row['driving_license'] == 0\
                                                 else (row['vehicle_damage'] + row['driving_license'])**2, axis=1)
        
        # vehicle_damage_vehicle_age
        df['vehicle_damage_vehicle_age'] = df.apply(lambda row: -(row['vehicle_damage'] + row['vehicle_age'])**2\
                                                    if row['vehicle_damage'] == 1\
                                                    else (row['vehicle_damage'] + row['vehicle_age'])**2, axis=1)

        # age / mean_age_by_region_code
        region_code_mean_age = pd.DataFrame(df.groupby('region_code')['age'].mean()).\
                                                    reset_index().rename(columns={'age':'mean_age_by_region_code'})
        
        df = pd.merge(df, region_code_mean_age, how='left', on='region_code')
        df['age_mean_age_by_region_code'] = df['age'] / df['mean_age_by_region_code']
        #del region_code_mean_age

        # age / mean_age_by_policy_sales_channel
        policy_sales_channel_mean_age = pd.DataFrame(df.groupby('policy_sales_channel')['age'].mean()).\
                                                    reset_index().rename(columns={'age':'mean_age_by_policy_sales_channel'})
        
        df = pd.merge(df, policy_sales_channel_mean_age, how='left', on='policy_sales_channel')
        df['age_mean_age_by_policy_sales_channel'] = df['age'] / df['mean_age_by_policy_sales_channel']
        #del policy_sales_channel_mean_age

        # annual_premium / mean_annual_premium_by_region_code
        region_code_mean_annual_premium = pd.DataFrame(df.groupby('region_code')['annual_premium'].mean()).\
                                                    reset_index().rename(columns={'annual_premium':'mean_annual_premium_by_region_code'})
        
        df = pd.merge(df, region_code_mean_annual_premium, how='left', on='region_code')
        df['annual_premium_mean_annual_premium_by_region_code'] = df['annual_premium'] / df['mean_annual_premium_by_region_code']
        #del region_code_mean_annual_premium

        # annual_premium / mean_annual_premium_by_policy_sales_channel
        policy_sales_channel_mean_annual_premium = pd.DataFrame(df.groupby('policy_sales_channel')['annual_premium'].mean()).\
                                                    reset_index().rename(columns={'annual_premium':'mean_annual_premium_by_policy_sales_channel'})
        
        df = pd.merge(df, policy_sales_channel_mean_annual_premium, how='left', on='policy_sales_channel')
        df['annual_premium_mean_annual_premium_by_policy_sales_channel'] = df['annual_premium'] / df['mean_annual_premium_by_policy_sales_channel']
        #del policy_sales_channel_mean_annual_premium

        # age / vintage
        df['age_vintage'] = (df['age']*365) / df['vintage']

        
        return df
    
    def data_preparation(self, df):
        # age_stage encoder
        df['age_stage'] = self.age_stage_encoder.fit_transform(df['age_stage'])
        
        # gender encoder
        df['gender'] = df['gender'].apply(lambda row: 0 if row == 'Male' else 1)
        
        return df

In [33]:
pipeline = insuranse_cross_sell()

In [40]:
df_teste.isnull().sum()

id                      0
Gender                  0
Age                     0
Driving_License         0
Region_Code             0
Previously_Insured      0
Vehicle_Age             0
Vehicle_Damage          0
Annual_Premium          0
Policy_Sales_Channel    0
Vintage                 0
dtype: int64

In [41]:
df01 = pipeline.data_cleaning(df_teste)
df02 = pipeline.feature_engineering(df01)
df03 = pipeline.data_preparation(df02)

In [42]:
df03.head().T

,0,1,2,3,4
id,381110.000000,381111.000000,381112.000000,381113.000000,381114.000000
gender,0.000000,0.000000,0.000000,0.000000,0.000000
age,25.000000,40.000000,47.000000,24.000000,27.000000
driving_license,1.000000,1.000000,1.000000,1.000000,1.000000
region_code,11.000000,28.000000,28.000000,27.000000,28.000000
previously_insured,1.000000,0.000000,0.000000,1.000000,1.000000
vehicle_age,0.000000,1.000000,1.000000,0.000000,0.000000
vehicle_damage,0.000000,1.000000,1.000000,1.000000,0.000000
annual_premium,35786.000000,33762.000000,40050.000000,37356.000000,59097.000000
policy_sales_channel,152.000000,7.000000,124.000000,152.000000,152.000000


In [43]:
df03.isnull().sum()

id                                                            0
gender                                                        0
age                                                           0
driving_license                                               0
region_code                                                   0
previously_insured                                            0
vehicle_age                                                   0
vehicle_damage                                                0
annual_premium                                                0
policy_sales_channel                                          0
vintage                                                       0
age_stage                                                     0
vehicle_damage_license                                        0
vehicle_damage_vehicle_age                                    0
mean_age_by_region_code                                       0
age_mean_age_by_region_code             